<a href="https://colab.research.google.com/github/Joyschool/gachon-discretemath/blob/main/11%EC%A3%BC%EC%B0%A8_%ED%8A%B8%EB%A6%AC_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.트리

## 9-0.트리 개요

- (코랩) 그래프에서 한글 폰트 사용 (실행 후-> 런타임 ->세션 다시 시작)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

- 공통 라이브러리

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
fontname = 'NanumGothic'          # (코랩)한글 폰트
plt.rc('figure', figsize = (6,4))
plt.rc('font', family=fontname)

- anytree - 트리를 쉽게 표현하는 라이브러리

In [ ]:
!pip install anytree

In [ ]:
# 트리 구조 만들기
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image

# 트리 구조 생성
v4 = Node('4')
v2 = Node('2', parent=v4)
v5 = Node('5', parent=v4)
v6 = Node('6', parent=v4)
v1 = Node('1', parent=v2)
v3 = Node('3', parent=v2)
v8 = Node('8', parent=v5)
v7 = Node('7', parent=v6)
v9 = Node('9', parent=v7)
v10 = Node('10', parent=v7)

# 1.트리 출력하기
print('1.Tree hierarchy')
for pre, fill, node in RenderTree(v4):
    print("%s%s" % (pre, node.name))

# 2.트리 출력하기
print('2.Tree hierarchy')
for line in DotExporter(v4):
    print(line)

# 3.트리 출력하기(이미지 파일)
print('3.Tree hierarchy')
DotExporter(v4).to_picture('test.png')
display(Image(filename="test.png"))

## 그래프(트리) 응용

### @자연어 처리 : 구문 분석 트리**
    - 1. jdk 설치
    - 2. Konlpy (Korean Natural Language Processing Library) 설치
        - 한국어 자연어 처리를 위한 파이썬 라이브러리
        - 형태소 분석기, 품사 태깅, 단어 추출 등 한국어 텍스트 처리에 필요한 다양한 기능 제공

In [ ]:
!apt-get update
!apt-get install -y openjdk-8-jdk

In [ ]:
!java -version

In [ ]:
!pip install konlpy

- [참고] 품사 태그 종류: https://happygrammer.github.io/nlp/postag-set/

In [ ]:
from konlpy.tag import Kkma
from nltk import Tree

# Konlpy의 Kkma 형태소 분석기 사용
kkma = Kkma()

# 한글 입력 문장
sentence = "자연어 처리는 컴퓨터와 인간의 언어 상호작용을 다루는 인공지능의 한 분야입니다."
# sentence = '''6411번 버스라고 있습니다. 서울시 구로구 가로수 공원에서 출발해서 강남을 거쳐서 개포동 주공 2단지까지 대략 2시간 정도 걸리는 노선버스입니다.
# 내일 아침에도 이 버스는 새벽 4시 정각에 출발합니다. 새벽 4시에 출발하는 그 버스와 4시 5분경에 출발하는 그 두 번째 버스는 출발한 지 15분 만에 신도림과 구로 시장을 거칠 때쯤이면 좌석은 만석이 되고 버스 사이 그 복도 길까지 사람들이 한 명 한 명 바닥에 다 앉는 진풍경이 매일 벌어집니다.'''

# 형태소 분석 및 구문 분석
parsed = kkma.pos(sentence)

# 형태소 분석 결과를 기반으로 트리 구조 생성
def build_tree(parsed):
    seen = set()  # 중복 제거를 위한 집합
    s_tree = Tree("S", [])  # 문장(S)
    np_tree = Tree("NP", [])  # 명사구(NP)
    vp_tree = Tree("VP", [])  # 동사구(VP)

    for word, pos in parsed:  # word:단어, pos: 품사 태그(Part-Of-Speech Tag)
        if (word, pos) in seen:  # 이미 처리된 단어/품사 조합은 스킵
            continue
        seen.add((word, pos))  # 새로운 단어/품사 조합 추가

        if pos.startswith("N"):  # 명사 관련 태그
            np_tree.append(f"{word}/{pos}")
        elif pos.startswith("V"):  # 동사 관련 태그
            vp_tree.append(f"{word}/{pos}")
        else:  # 기타 (조사, 부사 등)
            s_tree.append(f"{word}/{pos}")

    # 문장 트리에 명사구와 동사구 추가
    if np_tree:
        s_tree.append(np_tree)
    if vp_tree:
        s_tree.append(vp_tree)

    return s_tree

# 방법1: 트리 생성 및 트리 시각화(터미널에서 계층적 텍스트)
tree = build_tree(parsed)
print("구문 분석 트리 (텍스트 형태):\n", tree.pprint())

In [ ]:
# 트리 시각화: Networkx 그래프
def plot_tree(parsed):
    """
    품사 태깅 결과를 트리로 시각화
    - parsed: 형태소 분석 결과 [(단어, 품사), ...]
    """
    # 그래프 초기화
    G = nx.DiGraph()

    # 루트 노드 추가
    root = "ROOT"
    G.add_node(root)

    # 품사 태그별 트리 구성
    for word, pos in parsed:
        if pos.startswith("N"):  # 명사
            G.add_edge(root, f"{word}/{pos}")
        elif pos.startswith("V"):  # 동사
            G.add_edge(root, f"{word}/{pos}")
        else:  # 기타 품사
            G.add_edge(root, f"{word}/{pos}")

    # 그래프 시각화
    pos = nx.spring_layout(G, k=2)  # 노드 배치 자동 레이아웃
    # pos = nx.nx_pydot.graphviz_layout(G, prog="dot") # 트리구조(계층적) 레이아웃
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos,with_labels=True, node_size=1000, node_color="lightblue",
                font_size=8, font_weight="bold",arrowsize=12, font_family=fontname)
    plt.title("구문 분석 트리", fontsize=16)
    plt.title("구문 분석 트리", fontsize=16)
    plt.show()

# 품사 태깅 결과를 네트워크 트리로 시각화
plot_tree(parsed)

In [ ]:
# 형태소 분석 결과에서 명사만 추출
def extract_nouns(parsed):
    nouns = [word for word, pos in parsed if pos.startswith("N")]
    return nouns

# 명사 추출
nouns = extract_nouns(parsed)
print("추출된 명사: ", nouns)

### @지식 그래프 : 지식 추론
- 지식을 데이터로 정의하고 추론하기 위해 그래프 구조가 사용됨

In [ ]:
# 주어진 관계를 기반으로 특정 노드에서 추론된 노드를 반환
def infer(graph, start_node, relation):
    inferred_nodes = []
    for u, v, data in graph.edges(data=True):
        if u == start_node and data["relation"] == relation:
            inferred_nodes.append(v)
    return inferred_nodes

# 지식 추론
def get_infer_knowledge(start_node, relation_chain):
    current_nodes  = [start_node]
    for rel in relation_chain:
        next_nodes = []
        for node in current_nodes:
            next_nodes.extend(infer(G, node, rel))
        current_nodes = next_nodes

    return current_nodes

# 그래프 시각화
def plot_graph(G):
    # 쌍방향 간선 여부 체크
    def find_bidirectional_edges(G):
        bidirectional_edges = []
        for u, v in G.edges():
            if G.has_edge(v, u):  # (v, u)도 존재하면 추가
                bidirectional_edges.append((u, v))
        return bidirectional_edges

    if find_bidirectional_edges(G) : # 쌍방향 간선이 존재하면
        pos = nx.circular_layout(G)  # 원형 레이아웃
        nx.draw(G, pos,with_labels=True, node_size=1000, node_color="lightblue", connectionstyle="arc3,rad=0.2",
            font_size=8, font_weight="bold",arrowsize=14, font_family=fontname)
    else:
        # pos = nx.spring_layout(G, k=2)  # 노드배치: k 값을 크게 하면 엣지 길이가 늘어남
        pos = nx.shell_layout(G)
        pos = nx.nx_pydot.graphviz_layout(G, prog="dot")  # 노드 배치: 계층적으로 표시
        nx.draw(G, pos,with_labels=True, node_size=1000, node_color="lightblue",
                font_size=8, font_weight="bold",arrowsize=12, font_family=fontname)
    edge_labels = nx.get_edge_attributes(G, "relation")
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, font_family=fontname)
    plt.show()



# 테스트 데이터
nodes = [
    "서울", "대한민국", "동아시아",
    "뉴욕", "미국", "북아메리카",
    "파리", "프랑스", "유럽",
]
edges = [
    ("서울", "대한민국", "is_in"),
    ("대한민국", "동아시아", "is_in"),
    ("뉴욕", "미국", "is_in"),
    ("미국", "북아메리카", "is_in"),
    ("파리", "프랑스", "is_in"),
    ("프랑스", "유럽", "is_in"),
    ("대한민국", "서울", "capital_of"),
    ("미국", "뉴욕", "has_city"),
    ("프랑스", "파리", "capital_of"),
]

# 지식 그래프 생성
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from((u, v, {"relation": r}) for u, v, r in edges)

# 지식 추론 --> '서울'이 포함된 대륙 추론
start_node = "서울"
target_relation = ["is_in", "is_in"]  # '서울 -> 대한민국 -> 동아시아'
target_relation = ["is_in"]           # '서울 -> 대한민국
inferred_knowledge = get_infer_knowledge(start_node, target_relation)

# 추론 결과
print(f"'{start_node}'(이)가 '{target_relation}' 관계로 추론된 지식: {inferred_knowledge}")
print(f"'{start_node}'(이)가 포함된 대륙은: {inferred_knowledge}")

# 그래프 시각화
plot_graph(G)   # 그래프 그리기

- **지식추론을 위한 트리 탐방**: 탐색 경로상 모든 노드 탐색


In [ ]:
# 그래프(트리) 탐방 (DFS 또는 BFS)로 지식 추론
def traverse_and_infer(graph, start_node, target_relations):
    """
    트리 탐방을 사용하여 특정 노드에서 출발해 여러 관계를 기반으로 지식을 추론
    - graph: 방향성 있는 지식 그래프
    - start_node: 탐색을 시작할 노드
    - target_relations: 추론할 관계의 리스트
    """
    visited = set()  # 방문한 노드를 저장
    result = []  # 추론된 지식 저장

    # 깊이 우선 탐색 (DFS) 스택
    stack = [(start_node, None)]  # (노드, 관계)

    while stack:
        current_node, relation = stack.pop()
        if current_node not in visited:
            visited.add(current_node)
            # 주어진 관계가 타겟 관계 목록에 포함되면 결과에 추가
            if relation in target_relations:
                result.append(current_node)
            # 현재 노드와 연결된 모든 이웃을 스택에 추가
            for neighbor in graph.successors(current_node):
                stack.append((neighbor, graph[current_node][neighbor]["relation"]))

    return result


# 지식 데이터
nodes = [
    "동물", "조류", "포유류", "새", "펭귄", "박쥐", "고래",
    "고양이", "날다", "헤엄치다", "걷다"
]
edges = [
    ("동물", "조류", "is_a"),
    ("동물", "포유류", "is_a"),
    ("조류", "새", "is_a"),
    ("새", "펭귄", "is_a"),
    ("포유류", "박쥐", "is_a"),
    ("포유류", "고래", "is_a"),
    ("포유류", "고양이", "is_a"),
    ("새", "날다", "can"),
    ("펭귄", "헤엄치다", "can"),
    ("박쥐", "날다", "can"),
    ("고래", "헤엄치다", "can"),
    ("고양이", "걷다", "can"),
]

# 지식 그래프 생성
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from((u, v, {"relation": r}) for u, v, r in edges)

# 시작 노드와 추론 관계 설정
start_node = "새"
target_relations = [["can"], ["is_a","can"]]

# 지식 추론 실행
for target_relation in target_relations:
    inferred_knowledge = traverse_and_infer(G, start_node, target_relation)
    print(f"'{start_node}'가 '{target_relation}' 관계로 추론된 지식: {inferred_knowledge}")

# 그래프 시각화
plot_graph(G)



---



## 9-1.트리의 기본 개념

### [예제 9-1] 트리 판단하기
트리는 **사이클이 없는 연결 그래프(Acyclic Connected Graph)**로 정의됨
- (T, v0):  v0 내차수는 0, 나머지 정점의 내차수는 1(유향그래프일경우)
- v0 유일한 루트 노드
- T에는 어떤 순환도 존재하지 않는다.


In [ ]:
# pydot 그래프 모듈설치(이미 설치되었으면 skip)
!pip install pydot

In [ ]:
plt.rc('figure', figsize = (5,3))

# 그래프 생성하기
def create_directed_graph(edges): # 방향(유향) 그래프
    G = nx.DiGraph()
    G.add_edges_from(edges)
    return G

def create_graph(edges):          # 무향 그래프
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

def create_weighted_graph(edges): # 가중치 그래프
    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    return G

# 그래프 그리기: 트리 구조
def draw_graph_tree(G):
    # 노드 배치: 계층적으로 표시
    pos = nx.nx_pydot.graphviz_layout(G, prog="dot")
    nx.draw(G,pos,with_labels=True,
        node_size=1000, node_color="lightblue", font_size=8,font_weight="bold")
    plt.title("Graph: Tree", fontsize=10)
    plt.show()


# 트리 판단
def is_tree(edges):
    """
    그래프가 주어진 트리의 성질을 만족하는지 판단:
    1. 사이클 없음 (비순환)
    2. 하나의 고유한 루트 노드 존재
    3. 루트 노드의 내차수는 0이고, 나머지 노드의 내차수는 1
    """
    G = create_directed_graph(edges) # 유향 그래프 생성
    draw_graph_tree(G) # 그래프 그리기

    # 조건 1: 사이클 없음
    if not nx.is_directed_acyclic_graph(G):
        return False, "사이클이 존재합니다."

    # 조건 2: 루트 노드가 유일해야 함
    root_nodes = [node for node in G.nodes if G.in_degree(node) == 0]
    if len(root_nodes) != 1:
        return False, f"루트 노드가 {len(root_nodes)}개입니다."

    # 조건 3: 내차수 조건 확인
    for node in G.nodes:
        if node in root_nodes:  # 루트 노드
            if G.in_degree(node) != 0:
                return False, f"루트 노드 {node}의 내차수가 0이 아닙니다."
        else:  # 나머지 노드
            if G.in_degree(node) != 1:
                return False, f"노드 {node}의 내차수가 1이 아닙니다."

    # 모든 조건을 만족
    return True, "트리의 성질을 만족합니다."


# 테스트 데이터
a = [('a','c'),('b','c'),('c','d'),('c','e')]
b = [('a','b'),('a','c'),('b','d'),('b','e'),('c','f')]
c = [('a','b'),('a','c'),('b','d'),('b','e'),('c','f'),('c','g'),('g','a')]

# 트리 판별 실행
print(f"(a): {is_tree(a)} , {a}")
print(f"(b): {is_tree(b)} , {b}")
print(f"(c): {is_tree(c)} , {c}")

### [예제 9-2] 루트 노드 찾기
- A={v1, v2, v3, …, v10} 이고
- T={(v2, v1), (v2, v3), (v4, v5), (v4, v6), (v5, v8), (v6, v7), (v4, v2), (v7, v9), (v7, v10)} 일 때
- T는 트리가 되는데, 그 트리의 루트 노드를 구하라.    

In [ ]:
# 트리에서 루트 노드 찾기
def find_root_node(G):
    if isinstance(G, nx.DiGraph):  # 방향성 있는 그래프
        root_nodes = [node for node in G.nodes if G.in_degree(node) == 0]
    else:
        root_nodes = list(G.nodes)[0]  # 임의의 노드 선택

    # 트리의 조건: 루트 노드는 유일해야 함
    if len(root_nodes) == 1:
        return root_nodes[0]
    elif len(root_nodes) > 1:
        return f"루트 노드가 여러 개 존재합니다: {root_nodes}"
    else:
        return "루트 노드가 존재하지 않습니다."

# 테스트 데이터
A = set([f'v{i}' for i in range(1, 11)])
T = {('v2','v1'),('v2','v3'),('v4','v5'),('v4','v6'),('v5','v8'),
     ('v6','v7'),('v4','v2'),('v7','v9'),('v7','v10')}

# 그래프 생성
G = create_directed_graph(T) # 유향 그래프 생성
draw_graph_tree(G)           # 그래프 그리기

# 루트 노드 찾기
root_node = find_root_node(G)
print("루트 노드:", root_node)

### [예제 9-3] 트리 레벨과 높이(유향 그래프)
다음 트리를 보고 노드 v1, v3, v4, v6, v7, v8의 레벨과 높이를 구하라.

In [ ]:
# 잎 노드(리프 노드) 찾기
def find_leaf_nodes(G):
    if isinstance(G, nx.DiGraph):  # 유향 트리
        return [node for node in G.nodes if G.out_degree(node) == 0]
    else:
        return [node for node in G.nodes if G.degree(node) == 1]


# 트리에서 특정 노드의 차수(자식수)
def get_node_degree_in_tree(G, node, root=None):
    if isinstance(G, nx.DiGraph):  # 유향 트리
        return len(list(G.successors(node)))  # 자식 노드 수
    else:  # 무향 트리
        if root is None:
            raise ValueError("무향 트리에서는 root를 지정해야 합니다.")
        return len([neighbor for neighbor in G.neighbors(node) if neighbor != root])


# 특정 노드의 모든 조상 노드 찾기
def find_ancestors(G, node):
    return list(nx.ancestors(G, node))


# 트리 레벨과 높이 구하기
def calculate_level_height(G, root, target_nodes):
    """
    트리에서 노드의 레벨과 높이를 계산
    - G: 방향성 있는 트리 그래프 (DiGraph)
    - root: 트리의 루트 노드
    - target_nodes: 레벨과 높이를 계산할 대상 노드 리스트
    """
    # 레벨 계산 (BFS를 사용)
    level = {}
    for node in nx.single_source_shortest_path_length(G, root):
        level[node] = nx.shortest_path_length(G, root, node)

    # 높이 계산 (DFS를 사용)
    def compute_height(node, parent=None):
        if isinstance(G, nx.DiGraph):    # 유향 트리
            if G.out_degree(node) == 0:  # 리프 노드일 경우 높이는 0
                return 0
        else:
            if G.degree(node) == 1 and node != root:
                return 0
        # 유향 그래프와 무향 그래프에 따라 자식 노드를 탐색
        children = G.successors(node) if isinstance(G, nx.DiGraph) else (n for n in G.neighbors(node) if n != parent)
        # 재귀적으로 높이를 계산
        return 1 + max(compute_height(child, node) for child in children)

    # 높이 계산
    height = {node: compute_height(node) for node in G.nodes}

    # 결과 반환
    result = {node: {"level": level.get(node, None), "height": height.get(node, None)} for node in target_nodes}
    return result

In [ ]:
# 테스트 데이터: 유향 그래프 데이터
T = [("v1", "v2"), ("v1", "v3"), ("v2", "v4"), ("v2", "v5"),
         ("v3", "v6"), ("v3", "v7"), ("v4", "v8"), ("v6", "v9")]
target_nodes = ["v1", "v3", "v4", "v6", "v7", "v8"]

# 그래프 생성
G = create_directed_graph(T) # (트리)유향 그래프 생성
draw_graph_tree(G)           # 그래프 그리기

# 루트 노드 찾기
root_node = find_root_node(G)
print("루트 노드:", root_node)

# 잎 노드 찾기
leaf_nodes = find_leaf_nodes(G)
print("잎 노드(리프 노드):", leaf_nodes)

# 레벨과 높이 계산
result = calculate_level_height(G, root_node, target_nodes)
for node, values in result.items():
    print(f"노드 {node}: 레벨 = {values['level']}, 높이 = {values['height']}")

# 특정 노드의 차수 (v6)
node_degree = get_node_degree_in_tree(G, "v6")
print("노드 v6의 차수:", node_degree)

# 특정 노드의 조상 노드 (v6)
ancestors_v6 = find_ancestors(G, "v6")
print("노드 v6의 조상 노드:", ancestors_v6)



### [예제] 트리 레벨과 높이(무향 그래프)

In [ ]:
# 테스트 데이터: 무향 그래프
T = [("A", "B"), ("A", "C"), ("B", "D"),
     ("C", "E"), ("C", "F"), ("F", "G")]

# 그래프 생성
G = create_graph(T) # 그래프(트리)  생성
draw_graph_tree(G)  # 그래프 그리기

# 루트 노드 찾기
root_node = find_root_node(G)
print("루트 노드:", root_node)

# 1.잎 노드 찾기
leaf_nodes = find_leaf_nodes(G)
print("잎 노드(리프 노드):", leaf_nodes)

# 2.트리 레벨과 높이 계산
result = calculate_level_height(G, root_node, root_node)
for node, values in result.items():
    print(f"노드 {node}: 레벨 = {values['level']}, 높이 = {values['height']}")

# 3.특정 노드의 차수
node_degree = get_node_degree_in_tree(G, "C", root_node)
print("노드 C의 차수:", node_degree)

# 4.특정 노드의 조상 노드
ancestors = find_ancestors(G, "G")
print("노드 G의 조상 노드:", ancestors)

### [예제 9-4] 트리 활용 : 8-코인 문제
- 방법1: 완전 탐색(Brute force)
- 방법2: 분할 정복(Divide and Conquer)
- [참고] https://iq.opengenus.org/fake-coin-problem/

알고리즘 설명

- [방법1] 완전 탐색(Brute force) : 탐색 데이터의 위치에 따라 시행 횟수(시간복잡도)가 달라질 수 있다.


In [ ]:
import random
real_coin = 1
fake_coin = 0

# 임의의 코인 조건 만들기
def randomize_coins(n):
    global coins

    coins = [real_coin] * (n - 1) + [fake_coin] * (1) # 코인 리스트 만들기
    random.shuffle(coins)  # coins 리스트 무작위 섞기
    print(f'- coins={coins}')
    return coins

# fake코인 찾기
def findFakeCoin(n):
    trycnt = 1
    for i in range(1,n):
        if coins[0]==coins[i]:
            pass
        elif coins[0]<coins[i]:
            return print(f'- try[{trycnt}]: 1th is the fake one')
        else:
            return print(f'- try[{trycnt}]: {i+1}th coin is the fake one')
        trycnt+=1

n = 8  # 코인의 갯수
for i in range(5):
    print(f'[{i+1}회]-------------------')
    randomize_coins(n)  # 코인 리스트 만들기
    findFakeCoin(n)     # find fake coin

- 방법2: 분할 정복(Divide and Conquer): DecisionTree Methon(결정트리 방식), </br>특정 고정 크기 입력의 경우 시행 횟수는 일정할 수 있다.

In [ ]:
import random
from graphviz import Digraph
from IPython.display import Image, display

def build_decision_tree_graphviz():
    """
    8-coins 문제의 결정 트리를 생성하고 Graphviz로 시각화합니다.
    """
    # Graphviz를 사용한 트리 생성
    tree = Digraph(format="png", engine="dot")
    tree.attr(dpi="300")
    tree.attr(rankdir="TB")  # 트리를 수직 방향으로 정렬

    # 첫 번째 비교
    tree.node("A", "비교 1: 3 vs 3\n(ABC?DEF)")
    tree.node("B", "왼쪽 무거움\n(ABC)")
    tree.node("C", "오른쪽 무거움\n(DEF)")
    tree.node("D", "무게 동일\n(GH)")
    tree.edge("A", "B", label="왼쪽\n(ABC>DEF)")
    tree.edge("A", "C", label="오른쪽\n(ABC<DEF)")
    tree.edge("A", "D", label="같음\n(ABC==DEF)")

    # 두 번째 비교 (왼쪽이 무거운 경우)
    tree.node("E", "비교 2: 1 vs 1 (왼쪽 그룹)\n(A?B)")
    tree.edge("B", "E")

    tree.node("F", "왼쪽 무거움\n(A)")
    tree.node("G", "오른쪽 무거움\n(B)")
    tree.node("H", "무게 동일\n(C)")
    tree.edge("E", "F", label="왼쪽\n(A>B)")
    tree.edge("E", "G", label="오른쪽\n(A<B)")
    tree.edge("E", "H", label="같음\n(A==B)")

    # 두 번째 비교 (오른쪽이 무거운 경우)
    tree.node("I", "비교 2: 1 vs 1 (오른쪽 그룹)\n(D?E)")
    tree.edge("C", "I")

    tree.node("J", "왼쪽 무거움\n(D)")
    tree.node("K", "오른쪽 무거움\n(E)")
    tree.node("L", "무게 동일\n(F)")
    tree.edge("I", "J", label="왼쪽\n(D>E)")
    tree.edge("I", "K", label="오른쪽\n(D<E)")
    tree.edge("I", "L", label="같음\n(D==E)")

    # 두 번째 비교 (무게가 동일한 경우)
    tree.node("M", "비교 2: 남은 2 코인 비교\n(G?H)")
    tree.edge("D", "M")

    tree.node("N", "왼쪽 무거움\n(G)")
    tree.node("O", "오른쪽 무거움\n(H)")
    tree.edge("M", "N", label="왼쪽\n(G>H)")
    tree.edge("M", "O", label="오른쪽\n(G<H)")

    return tree


def find_different_coin(coins):
    """
    8개의 코인 중 다른 무게를 가진 코인을 찾아내는 결정 트리 방식.
    - coins: 8개의 코인의 무게 리스트 (무게가 다른 코인은 무작위로 설정됨)
    """
    print("초기 코인 상태:", coins)

    # 첫 번째 비교: 3 vs 3
    group1, group2, remaining = coins[:3], coins[3:6], coins[6:]
    print(f"비교 1: {group1} vs {group2}")
    if sum(group1) > sum(group2):
        print("결과: 왼쪽 그룹이 무거움")
        # 두 번째 비교: 1 vs 1
        print(f"비교 2: {group1[0]} vs {group1[1]}")
        if group1[0] != group1[1]:
            print(f"결과: {'왼쪽' if group1[0] > group1[1] else '오른쪽'} 코인이 다름")
            return group1[0] if group1[0] > group1[1] else group1[1]
        else:
            print("결과: 마지막 코인이 다름")
            return group1[2]
    elif sum(group1) < sum(group2):
        print("결과: 오른쪽 그룹이 무거움")
        # 두 번째 비교: 1 vs 1
        print(f"비교 2: {group2[0]} vs {group2[1]}")
        if group2[0] != group2[1]:
            print(f"결과: {'왼쪽' if group2[0] > group2[1] else '오른쪽'} 코인이 다름")
            return group2[0] if group2[0] > group2[1] else group2[1]
        else:
            print("결과: 마지막 코인이 다름")
            return group2[2]
    else:
        print("결과: 무게 동일, 나머지 코인 비교")
        # 세 번째 비교: 나머지 2개 비교
        print(f"비교 3: {remaining[0]} vs {remaining[1]}")
        return remaining[0] if remaining[0] != remaining[1] else None

# 트리 생성
decision_tree = build_decision_tree_graphviz()

# 이미지를 파일로 저장하고 화면에 표시
decision_tree.render("8-coins-problem", format="png", cleanup=True)
display(Image(filename="8-coins-problem.png"))

# 랜덤 무게 코인 생성
coins = [10] * 7  # 정상 코인 무게 10
different_coin_index = random.randint(0, 7)
coins[different_coin_index] = 12  # 무게가 다른 코인 추가

# 결정 트리로 코인 찾기
result = find_different_coin(coins)
print(f"무게가 다른 코인은: {result}")


-------------------------------

## 9-2. 레이블을 갖는 트리와 최소 스패닝 트리

### @레이블을 갖는 트리(labeled tree)
- 각 정점을 점으로 표시하고 그 점 옆에 우리가 필요한 레이블을 붙인 트리
- 트리의 노드(혹은 엣지)에 고유한 **식별자(label)**가 부여된 트리
- 레이블:e.g. 이름, 번호, 또는 기타 데이터를 포함할 수 있음
- 응용
    - 파일 시스템(폴더 이름이 노드 레이블로 표현됨)
    - 표현 언어(구문 트리의 노드 레이블)
    - 의사 결정 트리(조건이나 결과가 노드 레이블)


In [ ]:
# 레이블을 갖는 트리 생성
def create_labeled_tree():
    """
    레이블을 갖는 트리 생성
    """
    tree = nx.DiGraph()  # 방향성 있는 트리 생성

    # 노드 추가 (노드와 레이블 부여)
    tree.add_node("A", label="Root Node")
    tree.add_node("B", label="Child 1")
    tree.add_node("C", label="Child 2")
    tree.add_node("D", label="Child 1.1")
    tree.add_node("E", label="Child 1.2")
    tree.add_node("F", label="Child 2.1")
    tree.add_node("G", label="Child 2.2")

    # 엣지 추가 (엣지와 레이블 부여)
    tree.add_edge("A", "B", label="Edge A-B")
    tree.add_edge("A", "C", label="Edge A-C")
    tree.add_edge("B", "D", label="Edge B-D")
    tree.add_edge("B", "E", label="Edge B-E")
    tree.add_edge("C", "F", label="Edge C-F")
    tree.add_edge("C", "G", label="Edge C-G")

    return tree

# 레이블을 갖는 트리 시각화
def draw_labeled_tree(tree):
    pos = nx.nx_pydot.graphviz_layout(tree, prog="dot") # 노드 배치: 계층적으로 표시
    node_labels = nx.get_node_attributes(tree, "label") # 노드 레이블
    edge_labels = nx.get_edge_attributes(tree, "label") # 엣지 레이블

    nx.draw(tree, pos, with_labels=True, labels=node_labels, node_color="lightblue", node_size=1000, font_size=10)
    nx.draw_networkx_edge_labels(tree, pos, edge_labels=edge_labels, font_color="red")
    plt.title("Labeled Tree")
    plt.show()

# 실행
if __name__ == "__main__":
    tree = create_labeled_tree()
    draw_labeled_tree(tree)


#### [알고리즘 9-1] $M^N$ 레이블을 갖는 트리로 구성

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def create_power_tree_with_labels(M, N, MAX=1e6):
    """
    M^N을 구하는 알고리즘을 트리 형태로 시각화
    - M: 밑(base)
    - N: 지수(exponent)
    - MAX: 결과값 상한
    """
    tree = nx.DiGraph()  # 방향성 있는 그래프 생성
    root_label = f"Start: M={M}, N={N}"
    tree.add_node(root_label, value=1)  # 루트 노드

    def build_tree(node, M, N):
        """
        Pseudo 코드의 논리를 따라 트리를 구성
        """
        if N > 0:
            # Positive exponent
            value = 1
            for i in range(1, N + 1):
                value *= M
                current_label = f"Step {i}: S = {value}"
                tree.add_node(current_label, value=value)
                tree.add_edge(node, current_label)
                node = current_label
                # Check for MAX
                if value > MAX:
                    error_label = f"ERROR: Result > MAX ({MAX})"
                    tree.add_node(error_label, value=None)
                    tree.add_edge(node, error_label)
                    return
        elif N == 0:
            # Zero exponent
            zero_label = "Result: S = 1 (M^0 = 1)"
            tree.add_node(zero_label, value=1)
            tree.add_edge(node, zero_label)
        else:
            # Negative exponent
            if M == 0:
                undefined_label = "ERROR: M=0 and N<0 (Undefined)"
                tree.add_node(undefined_label, value=None)
                tree.add_edge(node, undefined_label)
                return
            else:
                recip_value = 1 / M
                value = 1
                for i in range(1, abs(N) + 1):
                    value *= recip_value
                    current_label = f"Step {i}: S = {value:.5f}"
                    tree.add_node(current_label, value=value)
                    tree.add_edge(node, current_label)
                    node = current_label

    # 트리 빌드 시작
    build_tree(root_label, M, N)
    return tree

# 레이블을 갖는 트리를 시각화
def draw_power_tree(tree):
    # pos = nx.nx_pydot.graphviz_layout(tree, prog="dot")  # 노드 배치: 계층적으로 표시
    pos = nx.spring_layout(tree)
    node_labels = nx.get_node_attributes(tree, "value")  # 노드 레이블
    formatted_labels = {key: f"{key}\nValue={val}" for key, val in node_labels.items()}

    nx.draw(tree, pos, with_labels=True, labels=formatted_labels, node_size=2000, node_color="lightblue", font_size=8)
    nx.draw_networkx_edges(tree, pos, arrows=True)
    plt.title("Power Calculation Tree with Labels")
    plt.show()

# 실행
if __name__ == "__main__":
    M = 2  # 밑
    N = -3  # 지수
    M, N = 2, 5
    MAX = 1000  # 최대 값 제한
    power_tree = create_power_tree_with_labels(M, N, MAX)
    draw_power_tree(power_tree)


#### [알고리즘 9-2] 산술식 계산 레이블을 갖는 트리로 구성
(3 - (2 * x)) + ((x - 2) - (3 + x))

- Step1: 산술식 트리 노드 구성

In [ ]:
def build_expression_tree():
    """
    산술식을 트리 형태로 생성
    (3 - (2 * x)) + ((x - 2) - (3 + x))
    """
    tree = nx.DiGraph()

    # 노드 추가: 중심 연산자
    tree.add_node("root", label="+")  # 최상위 연산자 '+'

    # 첫 번째 부분식: 3 - (2 * x)
    tree.add_node("node1", label="-")
    tree.add_node("3", label="3")
    tree.add_node("node2", label="*")
    tree.add_node("2", label="2")
    tree.add_node("x", label="x")

    # 두 번째 부분식: (x - 2) - (3 + x)
    tree.add_node("node3", label="-")
    tree.add_node("node4", label="-")
    tree.add_node("x1", label="x")
    tree.add_node("2_1", label="2")
    tree.add_node("node5", label="+")
    tree.add_node("3_1", label="3")
    tree.add_node("x2", label="x")

    # 엣지 추가 (연결 관계 정의)
    # 첫 번째 부분식: 3 - (2 * x)
    tree.add_edge("root", "node1")
    tree.add_edge("node1", "3")
    tree.add_edge("node1", "node2")
    tree.add_edge("node2", "2")
    tree.add_edge("node2", "x")

    # 두 번째 부분식: (x - 2) - (3 + x)
    tree.add_edge("root", "node3")
    tree.add_edge("node3", "node4")
    tree.add_edge("node3", "node5")
    tree.add_edge("node4", "x1")
    tree.add_edge("node4", "2_1")
    tree.add_edge("node5", "3_1")
    tree.add_edge("node5", "x2")

    return tree


def draw_expression_tree(tree):
    """
    레이블을 갖는 산술식 트리를 시각화
    """
    pos = nx.nx_pydot.graphviz_layout(tree, prog="dot")  # 노드 배치: 계층적으로 표시
    labels = nx.get_node_attributes(tree, "label")  # 노드 레이블 가져오기

    nx.draw(tree, pos, with_labels=True, labels=labels, node_color="lightblue", node_size=1000, font_size=8)
    nx.draw_networkx_edges(tree, pos, arrows=True)
    plt.title("Arithmetic Expression Tree")
    plt.show()


# 실행
if __name__ == "__main__":
    tree = build_expression_tree()
    draw_expression_tree(tree)


- Step2: 산술식 계산

In [ ]:
# 산술식 계산
def evaluate_expression_tree(tree, node, variable_values):
    """
    산술식 트리를 순회하여 값을 계산
    - tree: 트리 그래프
    - node: 현재 노드
    - variable_values: 변수 값 딕셔너리 (예: {"x": 5})
    """
    label = tree.nodes[node]["label"]
    print(f'label: {label}')

    # 리프 노드 처리 (숫자나 변수)
    if label.isdigit():
        return int(label)
    elif label.isalpha():  # 변수 처리
        return variable_values[label]

    # 내부 노드 처리 (연산자)
    children = list(tree.successors(node))
    print(f'children: {children}')
    left_value = evaluate_expression_tree(tree, children[0], variable_values)
    right_value = evaluate_expression_tree(tree, children[1], variable_values)

    # 연산 수행
    if label == "+":
        return left_value + right_value
    elif label == "-":
        return left_value - right_value
    elif label == "*":
        return left_value * right_value
    elif label == "/":
        return left_value / right_value


# 산술식 계산
variable_values = {"x": 5}  # 변수 값 설정
result = evaluate_expression_tree(tree, "root", variable_values)
print(f"산술식 결과: {result}")


- anytree로 구현

In [ ]:
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image
import operator

def build_expression_tree():
    """
    주어진 산술식 (3 - (2 * x)) + ((x - 2) - (3 + x)) 를 트리 형태로 생성.
    """
    # 트리 노드 생성
    root = Node("+")  # 루트 노드

    # 왼쪽 서브트리
    minus1  = Node("-", parent=root)
    node_3_1= Node("3", parent=minus1)
    mul1    = Node("*", parent=minus1)
    node_2_1= Node("2", parent=mul1)
    node_x_1= Node("x", parent=mul1)

    # 오른쪽 서브트리
    minus2  = Node("-", parent=root)
    sub_minus = Node("-", parent=minus2)
    node_x_2= Node("x", parent=sub_minus)
    node_2_2= Node("2", parent=sub_minus)
    sub_plus= Node("+", parent=minus2)
    node_3_2= Node("3", parent=sub_plus)
    node_x_2= Node("x", parent=sub_plus)

    return root


def visualize_tree(tree, output_file="expression_tree.png"):
    """
    트리를 이미지로 시각화.
    - tree: 루트 노드
    - output_file: 저장할 파일 이름
    """
    DotExporter(tree).to_picture(output_file)
    display(Image(filename=output_file))


def evaluate_expression_tree(node, variable_values=None):
    """
    산술식 트리를 계산.
    - node: 트리의 루트 노드
    - variable_values: 변수 값 (예: {"x": 5})
    """
    if variable_values is None:
        variable_values = {}

    # 연산자 함수 매핑
    operators = {
        "+": operator.add,
        "-": operator.sub,
        "*": operator.mul,
        "/": operator.truediv,
    }

    # 리프 노드: 숫자 또는 변수 처리
    if not node.children:
        if node.name.isdigit():  # 숫자인 경우
            return int(node.name)
        elif node.name in variable_values:  # 변수인 경우
            return variable_values[node.name]
        else:
            raise ValueError(f"변수 {node.name}의 값이 제공되지 않았습니다.")

    # 내부 노드: 연산자 처리
    left_value = evaluate_expression_tree(node.children[0], variable_values)
    right_value = evaluate_expression_tree(node.children[1], variable_values)

    if node.name in operators:
        return operators[node.name](left_value, right_value)
    else:
        raise ValueError(f"알 수 없는 연산자: {node.name}")


def print_tree(tree):
    """
    트리를 텍스트로 출력.
    - tree: 루트 노드
    """
    for pre, fill, node in RenderTree(tree):
        print(f"{pre}{node.name}")


# 실행
if __name__ == "__main__":
    # 1. 트리 생성
    root = build_expression_tree()

    # 2. 트리 출력 (텍스트 형태)
    print("트리 출력 (텍스트 형태):")
    print_tree(root)

    # 3. 트리 시각화 (이미지로 저장)
    visualize_tree(root)

    # 4. 트리 계산
    variable_values = {"x": 5}  # 변수 x에 대한 값 설정
    result = evaluate_expression_tree(root, variable_values)
    print(f"\n산술식 결과 (x={variable_values['x']}): {result}")


### @스패닝 트리(Spanning tree)
- **모든 정점들이 연결 되어 있어야 하고 사이클을 포함해서는 안된다.**
- Spanning Tree = 스패닝 트리= 신장 트리
- 정점 n은 n-1개의 간선을 갖는다.
- **그래프G에서 스패닝 트리를 구성하는 대표적인 방법**
    - 너비 우선 스패닝 트리(breadth first spanning tree)
    - 깊이 우선 스패닝 트리(depth first spanning tree
- 프로그래밍할 때 노드의 입력 순서에 때라 스패닝 트리 탐색 순서 결과는 달라질 수 있다.

### [예제] 인접행렬을 이용하여 DFS 깊이 우선 스패닝 트리 탐색

In [ ]:
# DFS를 이용한 spanning tree(인접행렬 방식)
def st_dfs(vtx, adj, s, visited) :
    visited[s] = True               # 현재 정점 s를 visited에 추가함
    for v in range(len(vtx)) :      # 인접행렬
        if adj[s][v] != 0 :         # 모든 간선 (s,v)에 대해
            if visited[v]==False:   # v를 아직 방문하지 않았으면
                print("(", vtx[s], vtx[v], ")", end=' ')  # 간선 출력
                st_dfs(vtx, adj, v, visited)


vtx = ['U','V','W','X','Y']
edge= [[0,  1,  1,  0,  0],
       [1,  0,  1,  1,  0],
       [1,  1,  0,  0,  1],
       [0,  1,  0,  0,  0],
       [0,  0,  1,  0,  0]]

print('spanning tree(인접행렬 방식): ', end="")
st_dfs(vtx, edge, 0, [False]*len(vtx))
print()

### [예제 9-5] 깊이 우선 스패닝 트리 & 너비 우선 스패닝 트리
그래프를 깊이 우선 스패닝 트리와 너비 우선 스패닝 트리로 표현하라.

In [ ]:
def create_spanning_trees(G):
    """
    깊이 우선 및 너비 우선 스패닝 트리를 생성
    - G: 원본 그래프
    """
    # DFS 스패닝 트리
    dfs_edges = list(nx.dfs_edges(G, source="v1"))
    dfs_tree = nx.Graph()
    dfs_tree.add_edges_from(dfs_edges)

    # BFS 스패닝 트리
    bfs_edges = list(nx.bfs_edges(G, source="v1"))
    bfs_tree = nx.Graph()
    bfs_tree.add_edges_from(bfs_edges)

    return dfs_tree, bfs_tree


def draw_spanning_trees(G, title, spanning_tree, edge_color):
    """
    원본 그래프와 깊이 우선 및 너비 우선 스패닝 트리를 시각화
    - G: 원본 그래프
    - spanning_tree: 깊이 우선 스패닝 트리/너비 우선 스패닝 트리
    """
    # pos = nx.spring_layout(G)  # 노드 위치 고정
    pos = nx.nx_pydot.graphviz_layout(G, prog="dot")  # 노드 배치: 계층적으로 표시

    # 원본 그래프 그리기
    nx.draw(G, pos, with_labels=True,
            node_color="lightblue", node_size=1000, font_size=8)

    # 스패닝 트리 그리기
    nx.draw_networkx_edges(G, pos, edgelist=list(spanning_tree.edges), edge_color=edge_color, width=2)

    plt.title(f"Graph with {title}")
    plt.legend([f"{title}"], loc="upper right", fontsize=8 )
    plt.show()



# 실행
if __name__ == "__main__":

    edges = [
        ("v1", "v2"), ("v1", "v3"),
        ("v2", "v4"), ("v2", "v5"),
        ("v3", "v6"), ("v3", "v7"),
        ("v4", "v8"), ("v5", "v8"),
        ("v6", "v8"), ("v7", "v8")
    ]

    # 그래프 생성
    G = create_graph(edges)

    # DFS 및 BFS 스패닝 트리 생성
    dfs_tree, bfs_tree = create_spanning_trees(G)

    # 스패닝 트리 시각화
    draw_spanning_trees(G, 'DFS Spanning Tree', dfs_tree, 'red')
    draw_spanning_trees(G, 'BFS Spanning Tree', bfs_tree, 'blue')


### @최소 스패닝 트리(Minimal Spanning Tree)
- 모든 정점들이 연결 되어 있어야 하고 사이클을 포함해서는 안된다.
- **그래프의 최소 연결 부분을 갖는 트리(가중치 합이 최소)**
- 알고리즘:
    - **프림 알고리즘(Prim's Algorithm)**: 시작 노드에서 가장 비용이 적은 간선(최소 가중치값)을 추가하며 스패닝 트리를 확장.
    - **크루스칼 알고리즘(Kruskal's Algorithm)**: 간선을 가중치 기준으로 정렬 후 신장 트리를 구성.
    

### [예제] 프림 알고리즘을 사용한 최소 스패닝 트리
- 강의자료 참고

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from heapq import heappop, heappush


# 프림 알고리즘 직접 구현
def prim_algorithm(graph, start_node):
    """
    프림 알고리즘을 직접 구현하여 최소 스패닝 트리(MST)를 생성
    - graph: 네트워크 그래프(NetworkX)
    - start_node: MST의 시작 노드
    """
    mst_edges = []  # MST를 구성하는 간선
    visited = set() # 방문한 노드 집합
    min_heap = []   # 최소 힙

    # 시작 노드에서 연결된 모든 간선을 힙에 추가
    visited.add(start_node)
    for neighbor, weight in graph[start_node].items():
        heappush(min_heap, (weight['weight'], start_node, neighbor))

    while min_heap:
        # print(f'min_heap: {min_heap}')
        weight, from_node, to_node = heappop(min_heap)  # 가장 작은 가중치 간선 꺼내기
        # print(f'pop_heap: {weight, from_node, to_node}')

        # 이미 방문한 노드라면 스킵
        if to_node in visited:
            continue

        # MST에 간선 추가
        mst_edges.append((from_node, to_node, weight))
        visited.add(to_node)

        # 새로운 노드에서 연결된 간선을 힙에 추가
        for neighbor, edge_data in graph[to_node].items():
            if neighbor not in visited:
                heappush(min_heap, (edge_data['weight'], to_node, neighbor))

    return mst_edges


def calculate_mst_properties(mst_edges):
    """
    MST의 경로와 가중치 합을 계산
    - mst_edges: MST를 구성하는 간선 리스트
    """
    mst_path = [(from_node, to_node) for from_node, to_node, weight in mst_edges]
    mst_weight_sum = sum(weight for _, _, weight in mst_edges)
    return mst_path, mst_weight_sum


def draw_graph_and_mst(G, mst_edges):
    """
    원본 그래프와 최소 스패닝 트리(MST)를 시각화
    """
    # pos = nx.spring_layout(G, seed=42)  # 노드 배치 고정
    pos = nx.nx_pydot.graphviz_layout(G, prog="dot")  # 노드 배치: 계층적으로 표시

    # 원본 그래프 그리기
    nx.draw(G, pos, with_labels=True,
            node_color="lightblue", node_size=1000, font_size=8)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, "weight"))

    # MST 강조하여 그리기
    nx.draw_networkx_edges(G, pos, edgelist=mst_edges, edge_color="red", width=2)

    plt.title("Prim's Algorithm - Minimum Spanning Tree (Red)")
    plt.show()


# 실행
if __name__ == "__main__":

    edges = [
        ('A','B',25), ('A','D',12), ('B','C',10), ('B','E',15),
        ('C','G',16), ('D','E',17), ('D','F',37), ('E','F',14),
        ('E','G',19), ('F','G',42)
    ]

    # 가중치 그래프 생성
    G = create_weighted_graph(edges)

    # 프림 알고리즘으로 MST 생성
    mst = prim_algorithm(G, start_node="A")
    print('------')

    # MST 경로와 가중치 합 계산
    mst_path, mst_weight_sum = calculate_mst_properties(mst)
    print("MST Path:", mst_path)
    print("MST Weight Sum:", mst_weight_sum)

    # 그래프와 MST 시각화
    draw_graph_and_mst(G, mst)

### [예제 9-6] 프림 알고리즘을 이용한 MST 찾기
프림 알고리즘을 이용해 다음 그래프에서 최소 스패닝 트리를 찾아라

In [ ]:
# 실행
if __name__ == "__main__":
    edges = [
        ("A", "C", 15), ("A", "B", 8),
        ("B", "G", 10), ("B", "D", 21),
        ("C", "F", 17), ("F", "G", 25),
        ("F", "E", 29), ("G", "E", 23),
        ("D", "E", 27)
    ]

    # 가중치 그래프 생성
    G = create_weighted_graph(edges)

    # 프림 알고리즘으로 MST 생성
    mst = prim_algorithm(G, start_node="A")
    print('------')

    # MST 경로와 가중치 합 계산
    mst_path, mst_weight_sum = calculate_mst_properties(mst)
    print("MST Path:", mst_path)
    print("MST Weight Sum:", mst_weight_sum)

    # 그래프와 MST 시각화
    draw_graph_and_mst(G, mst)


- networkx.minimum_spanning_tree() 사용한 프림 알고리즘(Prim's Algorithm)

In [ ]:
def prim_minimum_spanning_tree(G, start_node):
    """
    프림 알고리즘을 사용하여 최소 스패닝 트리(MST) 생성
    - G: 가중치 그래프
    - start_node: 시작 노드
    """
    # MST 생성
    mst = nx.minimum_spanning_tree(G, algorithm="prim")
    return mst


def calculate_mst_properties_nx(mst):
    """
    MST의 경로와 가중치 합계 계산
    - mst: 프림 알고리즘으로 생성된 MST (NetworkX Graph)
    """
    # MST 경로 (간선 리스트)
    mst_edges = list(mst.edges(data=True))

    # MST 가중치 합계
    total_weight = sum(data["weight"] for _, _, data in mst_edges)

    # 경로 추출
    path = [(u, v) for u, v, data in mst_edges]

    return path, total_weight


# 실행
if __name__ == "__main__":

    # 프림 알고리즘으로 MST 생성
    prim_mst = prim_minimum_spanning_tree(G, start_node="A")
    prim_mst_edges = list(prim_mst.edges)
    prim_path, prim_total_weight = calculate_mst_properties_nx(prim_mst)
    print("Prim's MST Path:", prim_path)
    print("Prim's MST Total Weight:", prim_total_weight)

    # 그래프와 MST 시각화 (프림 알고리즘)
    print("\nVisualizing Prim's MST...")
    draw_graph_and_mst(G, prim_mst_edges)


- 크루스칼 알고리즘(Kruskal Algorithm): networkx.minimum_spanning_tree() 사용

In [ ]:
def kruskal_minimum_spanning_tree(G):
    """
    크루스칼 알고리즘을 사용하여 최소 스패닝 트리(MST) 생성
    - G: 가중치 그래프
    """
    # MST 생성
    mst = nx.minimum_spanning_tree(G, algorithm="kruskal")
    return mst

 # 실행
if __name__ == "__main__":

    # 크루스칼 알고리즘으로 MST 생성
    kruskal_mst = kruskal_minimum_spanning_tree(G)
    kruskal_mst_edges = list(kruskal_mst.edges)
    kruskal_path, kruskal_total_weight = calculate_mst_properties_nx(kruskal_mst)
    print("\nKruskal's MST Path:", kruskal_path)
    print("Kruskal's MST Total Weight:", kruskal_total_weight)

    # 그래프와 MST 시각화 (크루스칼 알고리즘)
    print("\nVisualizing Kruskal's MST...")
    draw_graph_and_mst(G, kruskal_mst_edges)

### [예제 9-7] 최소 스패닝 트리
프림 알고리즘을 이용해 다음 가중치 그래프에서 노드 A로부터의 최소 스패닝 트리와 노드 E로부터의 최소 스패닝 트리를 찾아라.

In [ ]:
if __name__ == "__main__":
    edges = [
        ("A", "C", 2), ("C", "E", 5), ("E", "G", 3),
        ("A", "B", 3), ("B", "D", 6), ("D", "H", 2),
        ("B", "C", 3), ("C", "F", 5),
        ("D", "E", 2), ("E", "F", 4),
        ("H", "G", 6), ("G", "F", 4)
    ]

    for start in ["A", "E"]:
        # 가중치 그래프 생성
        G = create_weighted_graph(edges)

        # [방법1] : heapq사용 프림 알고리즘으로 MST 생성
        mst = prim_algorithm(G, start_node="A")
        mst_path, mst_weight_sum = calculate_mst_properties(mst)
        print("MST Path:", mst_path)
        print("MST Weight Sum:", mst_weight_sum)
        print("\nVisualizing Prim's MST...")
        draw_graph_and_mst(G, mst)

        # # [방법2] : nx사용 프림 알고리즘으로 MST 생성
        # prim_mst = prim_minimum_spanning_tree(G, start_node=start)
        # prim_mst_edges = list(prim_mst.edges)
        # prim_path, prim_total_weight = calculate_mst_properties_nx(prim_mst)
        # print("Prim's MST Path:", prim_path)
        # print("Prim's MST Total Weight:", prim_total_weight)
        # print("\nVisualizing Prim's MST...")
        # draw_graph_and_mst(G, prim_mst_edges)

        # # [방법3] : nx사용 크루스칼 알고리즘으로 MST 생성
        # kruskal_mst = kruskal_minimum_spanning_tree(G)
        # kruskal_mst_edges = list(kruskal_mst.edges)
        # kruskal_path, kruskal_total_weight = calculate_mst_properties_nx(kruskal_mst)
        # print("\nKruskal's MST Path:", kruskal_path)
        # print("Kruskal's MST Total Weight:", kruskal_total_weight)
        # print("\nVisualizing Kruskal's MST...")
        # draw_graph_and_mst(G, kruskal_mst_edges)

-------------------------

## 9-3. 이진 트리와 트리 탐방 알고리즘


### @이진 트리(binary Tree)
- 각 노드의 최대 차수가 2인 트리

### [예제 9-8] 이진 트리 탐색

In [ ]:
# 테스트 데이터: 무향 그래프
T = [("A", "B"), ("B", "D"), ("B", "E"),("D", "H"),
     ("A", "C"), ("C", "F"), ("C", "G")
    ]

# 그래프 생성
G = create_graph(T) # 그래프(트리) 생성
draw_graph_tree(G)  # 그래프 그리기

# 루트 노드 찾기
root_node = find_root_node(G)
print("루트 노드:", root_node)

# 1.잎 노드 찾기
leaf_nodes = find_leaf_nodes(G)
print("잎 노드(리프 노드):", leaf_nodes)

# 2.트리 레벨과 높이 계산
result = calculate_level_height(G, root_node, root_node)
for node, values in result.items():
    print(f"노드 {node}: 레벨 = {values['level']}, 높이 = {values['height']}")

# 3.특정 노드의 차수
for node in G.nodes:
    node_degree = get_node_degree_in_tree(G, node, root_node)
    print(f"노드 {node}의 차수:", node_degree)

# 4.특정 노드의 조상 노드
ancestors = find_ancestors(G, "G")
print("노드 G의 조상 노드:", ancestors)

- 이진 트리 판별하기

In [ ]:
# 주어진 그래프가 이진 트리인지 판별하기
def is_binary_tree(graph):

    # 무향 그래프 처리
    if isinstance(G, nx.Graph):
        # 조건 1: 연결 그래프인지 확인
        if not nx.is_connected(G):
            return False

        # 조건 2: 사이클이 없어야 함
        try:
            nx.find_cycle(G)
            return False  # 사이클이 있으면 트리가 아님
        except nx.NetworkXNoCycle:
            pass

        # 조건 3: 각 노드의 자식 수가 2 이하인지 확인 (DFS 사용)
        # - 앞에서 사용한 함수 사용 가능: get_node_degree_in_tree(G, node, root=None)
        def dfs(node, parent):
            child_count = 0
            for neighbor in G.neighbors(node):
                if neighbor != parent:  # 부모 노드는 제외
                    if not dfs(neighbor, node):
                        return False
                    child_count += 1
                    if child_count > 2:  # 자식 노드가 2개 초과
                        return False
            return True

        # DFS를 시작할 루트 노드 (아무 노드나 선택 가능)
        root = list(G.nodes)[0]
        if not dfs(root, None):
            return False

        return True

    # 유향 그래프 처리
    elif isinstance(G, nx.DiGraph):
        # 조건 1: 모든 노드의 자식 수가 2 이하인지 확인
        for node in G.nodes:
            if G.out_degree(node) > 2:  # 자식 노드 수가 2 초과
                return False

        # 조건 2: 사이클이 없어야 함
        try:
            nx.find_cycle(G)
            return False  # 사이클이 있으면 트리가 아님
        except nx.NetworkXNoCycle:
            pass

        # 조건 3: 루트 노드가 하나여야 함
        root_nodes = [node for node in G.nodes if G.in_degree(node) == 0]
        if len(root_nodes) != 1:
            return False

        return True

# 이진 트리 판별
if is_binary_tree(G):
    print("이 그래프는 이진 트리입니다.")
else:
    print("이 그래프는 이진 트리가 아닙니다.")

### @완전 이진 트리(complete binary tree)
- 트리의 마지막 레벨을 제외한 모든 레벨의 노드의 차수가 2
- 마지막 레벨은 노드의 차수가 2가 아니어도 되지만 왼쪽에서 오른쪽으로 차례대로 채워진 트리



### @포화 이진 트리(perfect binary tree)

- 터미널 노드를 제외한 모든 노드가 2개의 자식 노드를 가진 트리
- 터미널 노드가 동일한 깊이 or 레벨을 갖는 트리

### @트리 탐방(Tree Traversal) 알고리즘


- **트리로 표현된 자료를 선형 순서로 표현하는 방법**
    - E.g. : **(A + B) * (C - D)**
    - **전위 표기법(prefix)**: 트리로 표현된 자료를 전위 탐방하여 찾은 자료들을 선형으로 나열
        - 일반 전위 표기법ordinary prefix : ＊(＋(A, B), －(C, D))
        - 케임브리지 폴란드식 표기법Cambridge polish: ＊(＋AB)(－CD))
        - 폴란드식 표기법polish : ＊＋AB－CD
        
    - **후위 표기법(postfix)**: 후위 탐방하여 찾은 자료들을 선형으로 나열
        - 역 폴란드식 표기법(reverse polish)이라고 하며, 피연산자를 먼저 쓰고 다음에 연산자를 쓰는 방법
        - A B＋ C D－＊
    - **중위 표기법(infix)**: 중위 탐방하여 찾은 자료들을 선형으로 나열
        - 이진 연산에서만 적당한 표현 방법으로 피연산자와 피연산자 사이에 연산자를 표현하는 방법
        - A＋B＊C－D


- **트리 탐방 알고리즘**: 트리에서 필요한 자료를 찾고자 할 때 사용되는 알고리즘
    - **전위 탐방(preorder)**: preorder traversal
    - **후위 탐방(postorder)**: postorder traversal
    - **중위 탐방(inorder)**: inorder traversal

### [예제 9-9] 트리 탐방
- [그림 9-9]에 있는 트리를 전위 탐방, 중위 탐방, 후위 탐방 방법으로 탐방해서 결과를 전위 표기법, 중위 표기법, 후위 표기법으로 표현하라.
- (A + B) * (C - D)

In [ ]:
from anytree import Node as AnyNode, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image, display


# 노드 클래스 (탐방용 트리)
class Node:
    def __init__(self, item):
        self.item = item
        self.left = None
        self.right = None


# 이진 트리 클래스
class BinaryTree:
    def __init__(self):
        self.root = None
        self.anytree_root = None  # 시각화를 위한 anytree 루트 노드

    # 전위 순회: VLR (Root -> Left -> Right)
    def preorder(self, node):
        if node is not None:
            print(node.item, '', end='') # 루트 방문
            self.preorder(node.left)     # 왼쪽 서브트리
            self.preorder(node.right)    # 오른쪽 서브트리

    # 중위 순회: LVR (Left -> Root -> Right)
    def inorder(self, node):
        if node is not None:
            if node.left:
                print('(', end='')  # 왼쪽 서브트리 괄호 시작
                self.inorder(node.left)
            print(node.item, '', end='') # 루트 방문
            if node.right:
                self.inorder(node.right)
                print(')', end='')  # 오른쪽 서브트리 괄호 닫기

    # 후위 순회: LRV (Left -> Right -> Root)
    def postorder(self, node):
        if node is not None:
            self.postorder(node.left)    # 왼쪽 서브트리
            self.postorder(node.right)   # 오른쪽 서브트리
            print(node.item, '', end='') # 루트 방문


    # 트리를 시각화 (anytree 활용)
    def visualize(self):
        if self.anytree_root is None:
            raise ValueError("시각화를 위해 먼저 AnyTree 트리를 생성해야 합니다.")
        DotExporter(self.anytree_root).to_picture("tree.png")
        display(Image(filename="tree.png"))

    # anytree 트리 생성
    def build_anytree(self, node, parent=None):
        anytree_node = AnyNode(node.item, parent=parent)  # anytree 노드 생성
        if parent is None:
            self.anytree_root = anytree_node  # 루트 노드 설정
        if node.left:
            self.build_anytree(node.left, anytree_node)  # 왼쪽 서브트리
        if node.right:
            self.build_anytree(node.right, anytree_node)  # 오른쪽 서브트리
        return anytree_node


# 산술식을 트리 형태로 구성
n1 = Node('*')  # 루트 연산자
n2 = Node('+')  # 왼쪽 서브트리 연산자
n3 = Node('-')  # 오른쪽 서브트리 연산자
n4 = Node('A')  # 왼쪽 서브트리 피연산자
n5 = Node('B')  # 왼쪽 서브트리 피연산자
n6 = Node('C')  # 오른쪽 서브트리 피연산자
n7 = Node('D')  # 오른쪽 서브트리 피연산자

# 순서에 맞게 노드 연결
tree = BinaryTree()
tree.root= n1
n1.left  = n2
n1.right = n3
n2.left  = n4
n2.right = n5
n3.left  = n6
n3.right = n7

# 탐방 결과 출력
print("전위 표기법 (Preorder): ", end='')
tree.preorder(tree.root)
print()

print("중위 표기법 (Inorder): ", end='')
tree.inorder(tree.root)
print()

print("후위 표기법 (Postorder): ", end='')
tree.postorder(tree.root)
print()

# anytree 트리 생성 및 시각화
tree.build_anytree(tree.root)  # anytree 트리 빌드
tree.visualize()  # 트리 시각화


### [예제9-10] 트리 탐방 알고리즘
- 대수식 ((a - b) * c - (d / e)) * f  를 트리로 표현하고, 트리에서 전위 탐방, 중위 탐방, 후위 탐방을 해서 전위 표기법, 중위 표기법, 후위 표기법으로 표현하라.

In [ ]:
# 트리 구성
# ((a - b) * c - (d / e)) * f
n1 = Node('*')  # 루트: 최종 곱셈
n2 = Node('-')  # 왼쪽 서브트리
n3 = Node('f')  # 오른쪽 서브트리 피연산자
n4 = Node('*')  # 왼쪽 서브트리의 첫 번째 곱셈
n5 = Node('-')  # a - b
n6 = Node('c')  # c
n7 = Node('/')  # d / e
n8 = Node('a')  # a
n9 = Node('b')  # b
n10 = Node('d') # d
n11 = Node('e') # e

# 트리 연결
tree = BinaryTree()
tree.root = n1
n1.left = n2
n1.right = n3
n2.left = n4
n2.right = n7
n4.left = n5
n4.right = n6
n5.left = n8
n5.right = n9
n7.left = n10
n7.right = n11

# 탐방 결과 출력
print("전위 표기법 (Preorder): ", end='')
tree.preorder(tree.root)
print()

print("중위 표기법 (Inorder): ", end='')
tree.inorder(tree.root)
print()

print("후위 표기법 (Postorder): ", end='')
tree.postorder(tree.root)
print()

# anytree 트리 생성 및 시각화
tree.build_anytree(tree.root)  # anytree 트리 빌드
tree.visualize()  # 트리 시각화

### [예제] 산술수식 계산 트리
- 트리 기반의 산술 계산에서는 **후위 탐방(Postorder Traversal) 방식이 가장 효과적**입니다.
- 이유는 후위 순회가 계산에 필요한 연산자와 피연산자의 처리를 자연스럽게 지원하기 때문
- ((10 - 5) * 2 - (3 / 2)) * 4

In [ ]:
class Node:
    def __init__(self, item):
        self.item = item
        self.left = None
        self.right = None


class ExpressionTree:
    def __init__(self):
        self.root = None

    # 전위 순회: VLR (Root -> Left -> Right)
    def preorder(self, node):
        if node is not None:
            print(node.item, '', end='')  # 루트 방문
            self.preorder(node.left)    # 왼쪽 서브트리
            self.preorder(node.right)   # 오른쪽 서브트리

    # 중위 순회: LVR (Left -> Root -> Right)
    def inorder(self, node):
        if node is not None:
            if node.left:
                print('(', end='')  # 괄호 시작
                self.inorder(node.left)
            print(node.item, '', end='')  # 루트 방문
            if node.right:
                self.inorder(node.right)
                print(')', end='')  # 괄호 닫기

    # 후위 순회: LRV (Left -> Right -> Root)
    def postorder(self, node):
        if node is not None:
            self.postorder(node.left)   # 왼쪽 서브트리
            self.postorder(node.right)  # 오른쪽 서브트리
            print(node.item, '', end='')  # 루트 방문

    # 후위 순회 기반 계산
    def calculate(self, node):
        if node is not None:
            if node.item.isdigit():  # 피연산자일 경우
                return int(node.item)
            else:  # 연산자일 경우
                left_value = self.calculate(node.left)  # 왼쪽 서브트리 값
                right_value = self.calculate(node.right)  # 오른쪽 서브트리 값

                # 연산자에 따른 계산
                if node.item == '+':
                    return left_value + right_value
                elif node.item == '-':
                    return left_value - right_value
                elif node.item == '*':
                    return left_value * right_value
                elif node.item == '/':
                    return left_value / right_value


# 트리 구성
# ((10 - 5) * 2 - (3 / 2)) * 4
n1 = Node('*')  # 루트: 최종 곱셈
n2 = Node('-')  # 왼쪽 서브트리
n3 = Node('4')  # 오른쪽 서브트리 피연산자
n4 = Node('*')  # 왼쪽 서브트리의 첫 번째 곱셈
n5 = Node('-')  # 10 - 5
n6 = Node('2')  # 곱할 값
n7 = Node('/')  # 3 / 2
n8 = Node('10') # 10
n9 = Node('5')  # 5
n10 = Node('3') # 3
n11 = Node('2') # 2

# 트리 연결
tree = ExpressionTree()
tree.root = n1
n1.left = n2
n1.right = n3
n2.left = n4
n2.right = n7
n4.left = n5
n4.right = n6
n5.left = n8
n5.right = n9
n7.left = n10
n7.right = n11

# 탐방 결과 출력
print("전위 순회 (Preorder): ", end='')
tree.preorder(tree.root)
print()

print("중위 순회 (Inorder): ", end='')
tree.inorder(tree.root)
print()

print("후위 순회 (Postorder): ", end='')
tree.postorder(tree.root)
print()

# 계산 결과
result = tree.calculate(tree.root)
print("\n산술식의 계산 결과: ", result)


---------------------------

THE END